In [48]:
import math
from bitarray import *
limit = 1000000

In [2]:
# Code below adapted from https://cardinalpeak.com/blog/computing-primes-in-python-and-c/
Is_prime = bitarray(limit)
Is_prime.setall(True)
Is_prime[0] = False
Is_prime[1] = False
Is_prime[4:limit:2] = False
sqrt_limit = sqrt(limit)
i = 3

while i < sqrt_limit:
    Is_prime[i*i : limit : 2*i] = False
    try:
        i = (Is_prime.index(True, i + 1))
    except ValueError:
        break

In [3]:
print limit
print Is_prime.count()

1000000
78498


In [73]:
# Adapted from http://math.stackexchange.com/questions/5877/efficiently-finding-two-squares-which-sum-to-a-prime
# together with the arygcd code from NZMath at http://pydoc.net/Python/NZMATH/1.1.0/nzmath.arygcd/

"""
binary-like gcd algorithms for rational, gauss, and eisenstein integer
"""

def bit_num(a):
    """
    return the number of bits
    """
    if a == 0:
        return 1
    else:
        return int(math.log(a, 2)) + 1


def binarygcd(a, b):
    """
    Return the greatest common divisor of 2 integers a and b
    by binary gcd algorithm.
    """
    if a < b:
        a, b = b, a
    if b == 0:
        return a
    a, b = b, a % b
    if b == 0:
        return a
    k = 0
    while not a&1 and not b&1:
        k += 1
        a >>= 1
        b >>= 1
    while not a&1:
        a >>= 1
    while not b&1:
        b >>= 1
    while a:
        while not a & 1:
            a >>= 1
        if abs(a) < abs(b):
            a, b = b, a
        a = (a - b) >> 1
    return b << k


def arygcd_i(a1, a2, b1, b2):
    """
    Return the greatest common divisor of 2 gauss-integers a1+a2*i and b1+b2*i
    by (1+i)-ary gcd algorithm.
    """
    if a1 == 0 and a2 == 0:
       return b1, b2
    elif b1 == 0 and b2 == 0:
       return a1, a2
    ap, bp = 0, 0
    while (a1-a2)&1 == 0:
        a1, a2 = (a1+a2) >> 1, (a2-a1) >> 1
        ap += 1
    while (b1-b2)&1 == 0:
        b1, b2 = (b1+b2) >> 1, (b2-b1) >> 1
        bp += 1
    k = min(ap, bp)
    while a1 != 0 or a2 != 0:
        while (a1-a2) & 1 == 0:
            a1, a2 = (a1+a2) >> 1, (a2-a1) >> 1
        norm_a, norm_b = _ap_norm_i(a1, a2, b1, b2)
        if norm_a < norm_b:
            a1, a2, b1, b2 = b1, b2, a1, a2
        a1, a2 = _FormAdj_i(a1, a2)
        b1, b2 = _FormAdj_i(b1, b2)
        a1, a2 = a1-b1, a2-b2
    if 0 in (ap, bp):
        return b1, b2
    else:
        s, t = _n_pow_i(1, 1, k)
        return (b1*s)-(b2*t), (b1*t)+(b2*s)

def _ap_norm_i(a, b, c, d):
    """
    Return approximately norm of 2 gaussInteger a+b*i and c+d*i  
    """
    a, b, c, d = abs(a), abs(b), abs(c), abs(d)
    max_dig = max(bit_num(a), bit_num(b), bit_num(c), bit_num(d))
    if max_dig > 6:
        sft_num = max_dig - 6
        a, b, c, d = a >> sft_num, b >> sft_num, c >> sft_num, d >> sft_num
        return a*a + b*b, c*c + d*d
    else:
        return a*a + b*b, c*c + d*d

def _n_pow_i(a, b, n):
    """
    return (1+i)**k 
    """
    x = a
    y = b

    for i in range(1, n):
        x1 = (x*a) - (y*b)
        y1 = (y*a) + (x*b)
        x = x1
        y = y1
    return x, y

def _FormAdj_i(a, b):
    """
    transform gaussInteger a+b*i ->  form 1+2(1+i)*(x+y*i)
    """
    if a % 2 == 0:
        a, b = -b, a
    if (b - a + 1) % 4 == 0:
        return a, b
    else:
        return -a, -b


def arygcd_w(a1, a2, b1, b2):
    """
    Return the greatest common divisor of 2 eisensteinIntegers a1+a2*w and b1+b2*w
    by (1-w)-ary gcd algorithm.
    """
    if a1 == 0 and a2 == 0:
       return b1, b2
    elif b1 == 0 and b2 == 0:
       return a1, a2
    ap, bp = 0, 0
    while (a1 + a2) % 3 == 0:
        a1, a2 = (a1 + a1 - a2) / 3, (a1 + a2) / 3
        ap += 1
    while (b1 + b2) % 3 == 0:
        b1, b2 = (b1 + b1 - b2) / 3, (b1 + b2) / 3
        bp += 1
    k = min(ap, bp)
    while a1 != 0 or a2 != 0:
        while (a1 + a2) % 3 == 0:
            a1, a2 = (a1 + a1 - a2) / 3, (a1 + a2) / 3
        nrm_a, nrm_b = _ap_norm_w(a1, a2, b1, b2)
        if nrm_a < nrm_b:
            a1, a2, b1, b2 = b1, b2, a1, a2
        a1, a2 = _FormAdj_w(a1, a2)
        b1, b2 = _FormAdj_w(b1, b2)
        a1, a2 = a1 - b1, a2 - b2
    k1, k2 = _n_pow_w(k)
    return k1*b1 - k2*b2, k1*b2 + k2*b1 - k2*b2

def _ap_norm_w(a, b, c, d):
    """
    Return approximately norm of 2 eisensteinInteger a+b*w and c+d*w  
    """
    a, b, c, d = abs(a), abs(b), abs(c), abs(d)
    max_dig = max(bit_num(a), bit_num(b), bit_num(c), bit_num(d))
    if max_dig > 6:
        sft_num = max_dig - 6
        a, b, c, d = a >> sft_num, b >> sft_num, c >> sft_num, d >> sft_num
        subst1, subst2 = (a - b) >> sft_num, (c - d) >> sft_num
        return a*a + b*b + subst1*subst1, c*c + d*d + subst2*subst2
    else:
        return a*a + b*b + (a - b)*(a - b), c*c + d*d + (c -d)*(c -d)

def _n_pow_w(n):
    """
    return (1-w)**k
    """
    x, y = divmod(n, 2)
    k1 = 3**x
    if y == 1:
        k2 = -k1
    else:
        k2 = 0
    return k1, k2

def _FormAdj_w(a1, a2):
    """
    transform eisensteinInteger a1+a2*w ->  form 1+3*(x+y*w)
    """
    if a1 % 3 == 0:
        if a2 % 3 == -1 or a2 % 3 == 2:
            return a1 - a2, a1
        else:
            return a2 - a1, -a1
    elif a1 % 3 == 1:
        if a2 % 3 == 1:
            return a2, a2 -a1
        else:
            return a1, a2
    else:
        if a2 % 3 == -1 or a2 % 3 == 2:
            return -a2, a1 -a2
        else:
            return -a1, -a2

def root4(p):
    # 4th root of 1 modulo p
    if p <= 1:
        return "too small"
    if (p % 4) != 1:
        return "not congruent to 1"
    k = p/4
    j = 2
    while True:
        a = pow(j, k, p)
        b = (a * a) % p
        if b == p-1:
            return a
        if b != 1:
            return "not prime"
        j += 1

def root3(p):
    # 3rd root of 1 modulo p
    if p <= 1:
        return "too small"
    if (p%3) != 1:
        return "not congruent to 1"
    k = p/3
    j = 2
    while True:
        a = pow(j,k,p)
        b = pow(a,3,p)
        if b != 1:
            return "not prime"
        if a != 1:
            return a
        j += 1
        
def gaussfac(p):
    a = root4(p)
    return arygcd_i(p,0,a,1)

def eisenfac(p):
    a = root3(p)
    return arygcd_w(p,0,a,-1)

In [74]:
p = 97
a = root3(p)
print a, pow(a,3,p)


35 1


In [78]:
u,v = eisenfac(31)
print u,v
print u*u - u*v + v*v

-5 -6
31


In [216]:
alimit = 100000

In [217]:
gprimes = []
plist = [p for p in range(alimit) if ((p%4 == 1) and Is_prime[p])]
for p in plist:
    gprimes.append(gaussfac(p))
print len(plist)
print len(gprimes)

4783
4783


In [218]:
eprimes = []
qlist = [q for q in range(alimit) if ((q%3 == 1) and Is_prime[q])]
for q in qlist:
    eprimes.append(eisenfac(q))
print len(qlist)
print len(eprimes)

4784
4784


In [219]:
radius = 5
rout = radius * 1.02
largest = plist[-1]
sl = sqrt(largest)
dotradius = radius*0.5/ sl
S = ''
for i in range(len(plist)):
    p = plist[i] # A prime congruent to 1 mod 4.
    r = sqrt(p) # Its square root.
    g = gprimes[i] # A Gaussian prime of norm p
    # (ax, ay) on unit circle, at angle of g
    ax = g[0]/r
    ay = g[1]/r
    # (gx, gy) within unit circle.
    gx = g[0]/sl
    gy = g[1]/sl
    S += '\\fill[blue] (%.3f in, %.3f in) circle (%.3f in);'%(gx * radius, gy * radius, dotradius)
    S += '\\draw[black, opacity=0.1] (%.3f in, %.3f in) -- (%.3f in, %.3f in);'%(ax * radius, ay*radius, ax*rout, ay*rout)
    S += '\n'

pilist = [p for p in range(int(sl)) if ((p%4 == 3) and Is_prime[p])]
for p in pilist:
    gx = p/sl
    gy = 0
    S += '\\fill[red] (%.3f in, %.3f in) circle (%.3f in);'%(gx * radius, gy * radius, dotradius)
    S += '\n'
    
S += '\\draw[red] (%.3f in, %.3f in) -- (%.3f in, %.3f in);'%(1 * radius, 0, 1*rout, 0)   

p = 2
r = sqrt(p)
g = (1,1)
ax = g[0]/r
ay = g[1]/r
# (gx, gy) within unit circle.
gx = g[0]/sl
gy = g[1]/sl
S += '\\fill[green] (%.3f in, %.3f in) circle (%.3f in);'%(gx * radius, gy * radius, dotradius)
S += '\\draw[green] (%.3f in, %.3f in) -- (%.3f in, %.3f in);'%(ax * radius, ay*radius, ax*rout, ay*rout)
S += '\n'


In [220]:
f = open('GCircle.tex', 'w')
f.write(S)
f.close()

In [221]:
len(plist)

4783

In [222]:
sthree = sqrt(3) / 2

In [223]:
radius = 5
rout = radius * 1.02
largest = qlist[-1]
sl = sqrt(largest)
dotradius = radius*0.5 / sl
T = ''
for i in range(len(qlist)):
    q = qlist[i] # A prime congruent to 1 mod 3.
    r = sqrt(q) # Its square root.
    e = eprimes[i] # An Eiseinstein prime of norm p
    # (ax, ay) on unit circle, at angle of g
    ax = (e[0] - (e[1] * 0.5) )/r
    ay = e[1] * sthree /r
    # (gx, gy) within unit circle.
    ex = (e[0] - (e[1] * 0.5) )/sl
    ey = e[1] * sthree/sl
    T += '\\fill[blue] (%.3f in, %.3f in) circle (%.3f in);'%(ex * radius, ey * radius, dotradius)
    T += '\\draw[black, opacity=0.1] (%.3f in, %.3f in) -- (%.3f in, %.3f in);'%(ax * radius, ay*radius, ax*rout, ay*rout)
    T += '\n'
    
pilist = [p for p in range(int(sl)) if ((p%3 == 2) and Is_prime[p])]
for p in pilist:
    gx = p/sl
    gy = 0
    T += '\\fill[red] (%.3f in, %.3f in) circle (%.3f in);'%(gx * radius, gy * radius, dotradius)
    T += '\n'
    
T += '\\draw[red] (%.3f in, %.3f in) -- (%.3f in, %.3f in);'%(0.5 * radius, sthree * radius, 0.5*rout, sthree * rout)   

p = 3
r = sqrt(p)
e = (1,-1)
ax = (e[0] - (e[1] * 0.5) )/r
ay = e[1] * sthree /r
# (gx, gy) within unit circle.
ex = (e[0] - (e[1] * 0.5) )/sl
ey = e[1] * sthree/sl
T += '\\fill[green] (%.3f in, %.3f in) circle (%.3f in);'%(ex * radius, ey * radius, dotradius)
T += '\\draw[green] (%.3f in, %.3f in) -- (%.3f in, %.3f in);'%(ax * radius, ay*radius, ax*rout, ay*rout)

In [224]:
f = open('ECircle.tex', 'w')
f.write(T)
f.close()

In [225]:
print len(plist) * 8 + len(qlist) * 12

95672
